VCCs = Vulnerability-Contributing-Commits

## Vulnerability Lifecycle
![Vulnerability detection chart](Vulnerability_detection_chart.png)

### 🧩 Vulnerability Lifecycle Phases

- **Phase 1 — Introduction → Discovery (`t_int` → `t_f`):**  
  The vulnerability is introduced into a product (at time `t_int`) and remains undetected until it is discovered (at time `t_f`).  
  This phase ends when the vulnerability is first found by any party.

- **Phase 2 — Private Knowledge (`t_f` → `t_d`):**  
  The vulnerability is known to at least one individual or organization but has **not yet been publicly disclosed**.  
  During this period, exploitation risk depends on the discoverer’s intent (ethical researcher vs. malicious actor).

- **Phase 3 — Public Disclosure → Patch Release (`t_d` → `t_fix`):**  
  Begins once the vulnerability becomes **publicly known** (at `t_d`) and lasts until a **patch or fix** is published (at `t_fix`).  
  This phase can be eliminated if the vendor discloses and fixes the issue simultaneously.

- **Phase 4 — Patch Release → Full Remediation (`t_fix` → `t_p`):**  
  Covers the period after the fix is released until **all affected hosts have applied the patch** (at `t_p`).  
  The duration depends on users’ update behavior and deployment speed.

---

#### 🧠 Notes
- Phases can be **long, short, or even non-existent**, depending on how the vulnerability is handled.  
- In **responsible disclosure**, Phase 3 may be skipped entirely (disclosure and fix occur together).  
- In some **proprietary software**, vulnerabilities may be **silently patched**—public disclosure never occurs, so the lifecycle differs.

## Research Questions:

- How long do vulnerabilities remain in the code? 
- Do these lifetimes differ for different projects or different vulnerability types? 
- How long does it take to find certain portions of ultimately discovered vulnerabilities 

Answering these questions will provide us with insights regarding the duration of the window of exposure for different projects.

- Are lifetimes increasing or decreasing over time? 
- Are there signs of improved qual- ity? 
- How did the introduction of automated tools affect lifetimes? 

Answering these questions will help us approach a fundamental question of software security from a novel angle: is software getting more secure over time?




### 📊 Summary of Prior Work on Vulnerability Detection and Patching Behavior

| **Study / Authors** | **Lifecycle Phase(s)** | **Focus / Methodology** | **Key Findings** | **Remarks** |
|----------------------|------------------------|--------------------------|------------------|--------------|
| **Nappa et al. [74]** | Phase 4 (Patching speed vs. attacks) | Empirical study of patch deployment and attack timing | Only **14% of hosts** patched (median) when exploits became public | Demonstrated slow patch adoption rates |
| **Sarabi et al. [98]** | Phase 4 | Extended Nappa’s study with emphasis on **end-user behavior** | Analyzed how users’ actions affect patch uptake | Focused on behavioral factors |
| **Kotzias et al. [51]** | Phase 4 | Large-scale measurement of **enterprise vs. consumer hosts** | Enterprise hosts patched **faster**, but exposure window still significant | Showed organizational differences in patching speed |
| **Krebs [52, 53]; Frei [37]; Shahzad et al. [103]** | Phases 2 & 3 | Measured timing between **reporting, disclosure, exploit availability, and patch release** | **>80%** of vulnerabilities (2008–2011) patched **before public disclosure** | Studied industry-wide patch release behavior |
| **Bilge et al. [15]** | Phase 2 (Zero-day exploitation) | Analysis of **zero-day attack durations** in Windows | Zero-days lasted **19 days–30 months** (avg. **312 days**) | Quantified real-world exploitation duration |
| **Rescorla [94]** | Phases 1–3 (Vulnerability lifetime) | Used ICAT database to estimate vulnerability lifetimes | Mean lifetime **2.5 years**, roughly exponential distribution | Early large-scale empirical study; limited data quality |
| **Ozment & Schechter [84]** | Phases 1–3 (Foundational vulnerabilities in OpenBSD) | Manual inspection of version control history | Median lifetime **≥2.6 years**, declining discovery rate | Focused on long-lived, foundational flaws |
| **Corbet [26]** | Phases 1–3 (Linux kernel CVEs) | Small-scale study on **80 CVEs** (manual curation) | Provided visual data only, limited analysis | Raised community interest on vulnerability aging |
| **Cook [25]** | Phases 1–3 (Linux kernel CVEs) | Analyzed **557 CVEs** using Ubuntu Security Team data | Produced distribution plots, limited depth | Motivated further academic study |
| **Li & Paxson [58]** | Phases 1–3 (Open source projects) | Large-scale analysis of security patching in open source | Estimated **lower bounds** for lifetimes; lifetimes not correlated with type | Most comprehensive modern study; differs methodologically from later work |

---

#### 🧠 Notes
- **Phase 2** → Period when vulnerabilities are known privately (e.g., zero-days).  
- **Phase 3** → From public disclosure to patch release.  
- **Phase 4** → From patch release to full remediation (user adoption).  
- **Phases 1–3 combined** → Represent *vulnerability lifetime* in the codebase before a fix is published.

### 🧾 Key Definitions: Vulnerability Code Lifetime

- **Vulnerability Lifetime (Code Lifetime):**  
  The amount of time a vulnerability remains in the codebase.  
  It represents the period between:
  - The **introduction** of a weakness (via a code change), and  
  - The **fix** of that weakness (via a later code change).  

  In other words, it is the duration between the commit that introduces the vulnerability and the commit that removes or mitigates it.

---


- **Vulnerability Contributing Commit (VCC):**  
  A commit that **introduces or contributes to the introduction** of a vulnerability in the codebase.  
  There may be multiple VCCs for a single vulnerability, for example when:
  - The weakness is gradually introduced over multiple commits.
  - The vulnerability appears due to refactoring or feature extensions.

---

- **Fixing Commit:**  
  A commit that **removes, mitigates, or corrects** a previously introduced vulnerability.  
  Like VCCs, there may be multiple fixing commits if:
  - The vulnerability is patched in stages.
  - Different branches or versions of the software are fixed separately.

---

> **Summary:**  
> In essence, a vulnerability’s *lifetime* is measured as the elapsed time between its **VCC(s)** and **fixing commit(s)** — from introduction to resolution — as captured within a version control system.

- ##### CVE (Common Vulnerabilities and Exposures):
A unique ID assigned to a publicly known cybersecurity vulnerability.

- ##### NVD (National Vulnerability Database):
A U.S. government–maintained database that provides detailed information, metrics, and severity scores for CVEs.

- ##### FLOSS:
Free/Libre and Open Source Software

#### Some points:

- a CVE may cover multiple programming errors.
- a vulnerability may be removed and then re-introduced.
- a fix may be extensive, requiring multiple commits.

### ⚙️ Defining Start/End Points and Timestamps for Vulnerability Lifetime

#### **Start and End Points**
- Use the **first Vulnerability Contributing Commit (VCC)** as the **start** of the lifetime.  
- Use the **last fixing commit** as the **end** of the lifetime.  
- For vulnerabilities with **multiple sub-vulnerabilities** under one CVE, measure the **oldest** one.  
- For **re-introductions**, measure the **total lifetime**, as the risk persists for systems running stable versions that may not treat premature fixes as critical patches.

---

#### **Timestamps**
- Initially considered using the **merge time** into the main branch, since that’s when code becomes publicly available.  
- However, this adds complexity and offers little benefit — the time between commit and merge is short (≈ 20 days) compared to typical vulnerability lifetimes (years).  
- Therefore, the **commit timestamp** is used as the standard metric because:
  - It is consistently available across Git, SVN, and Mercurial.  
  - It avoids complications with unmerged fixes or branch-specific patches.

---

> **Final Decision:**  
> Measure vulnerability lifetime from the **first VCC’s commit timestamp** to the **last fixing commit’s timestamp**.

### Some important datasets:

- Ubuntu CVE Tracker - 824 mappings between CVEs and their VCCs
- Vulnerability History Project - 295 ground truth mappings for Chromium13, and 74 for the Apache HTTP Server (Httpd)

Overall they were able to collect mappings for 1193 CVEs to one or more VCCs, from high-quality sources. These mappings constitue our ground truth dataset.

### 🔹 Summary: Dataset and CVE-to-Commit Mappings

#### **Ground Truth Dataset**
- Mappings of **CVEs → Vulnerability Contributing Commits (VCCs)** form the ground truth.  
- Sourced from **manually curated datasets** by researchers and project maintainers.  
- Used to **evaluate and validate automated methods** for estimating vulnerability lifetimes.

#### **Sources of Mappings**
- **Ubuntu CVE Tracker:**  
  - Curates Linux kernel vulnerabilities with VCCs and fixing commits.  
  - After filtering early commits, 824 CVE-to-VCC mappings were used.  
- **Vulnerability History Project:**  
  - 295 mappings for Chromium, 74 for Apache HTTP Server.  
  - Focused on C/C++ repositories to ensure sufficient data.  
- **Total Ground Truth Dataset:**  
  - **1,193 CVEs** linked to one or more VCCs from high-quality sources.

#### **Project Selection Criteria for Analysis**
- Must be **FLOSS** (Free/Libre Open Source Software) with transparent security workflows.  
- Must have at least **100 CVEs** for statistical significance.  
- Mainly **C/C++ projects** (insufficient ground truth for other languages).  
- A significant portion of CVEs must be **linkable to fixing commits**.  
- Resulting dataset: **11 diverse projects**, varying in size, age, and domain.

#### **Linking CVEs to Fixing Commits**
- Based on **NVD (National Vulnerability Database)** data.  
- Four main approaches used to identify fixing commits:
  1. **CVE-ID in commit message:**  
     - Directly links commits mentioning CVE-ID (project-specific syntax verified manually).  
  2. **Commit in NVD references:**  
     - References to repository commits in NVD advisories; verified via sampling.  
  3. **Common Bug ID:**  
     - Link commits to NVD-referenced bug tracking IDs using regex.  
  4. **Third-party mappings:**  
     - Incorporates curated datasets from Linux Kernel CVEs, Piantadosi et al., and Debian Security Tracker.  
- Combined approach ensures **largest and highest-quality dataset** of CVE-to-fixing-commit mappings.

#### **Dataset Goal**
- Provide a **representative sample of FLOSS projects** with high-quality CVE-to-commit links.  
- Enable **robust analysis of vulnerability lifetimes** and validation of automated estimation methods.

### 🧩 Methods for Calculating Vulnerability Lifespan and Accuracy Comparison

- **Previous Approaches**
  - **Manual Identification**  
    Earlier studies manually traced Vulnerability Contributing Commits (VCCs) by analyzing fixing commits — highly accurate but not scalable for large datasets.  
  - **Li and Paxson’s Automated Approach**  
    Used the `git blame` command on modified or deleted lines in fixing commits to find potential VCCs.  
    - Selected the *most recent* commit among candidates → produced a **lower bound** estimate of vulnerability lifetime.  
    - **Shortcoming:** Underestimated average lifetimes by nearly **1 year (~347 days)** — too conservative.  
  - **VCCFinder (Perl et al.)**  
    Enhanced `git blame` by also analyzing *added lines* in fixing commits. The commit most frequently blamed was marked as the VCC.  
    - **Claimed Accuracy:** 96% (based on limited manual validation).  
    - **Empirical Accuracy:** ~40% (based on ground truth data), showing reduced reliability for precise lifetime estimation.

- **Key Insight**  
  Manual methods are accurate but not scalable; automated heuristics like `git blame` are scalable but tend to **underestimate or misidentify** VCCs, reducing their usefulness for precise vulnerability lifetime analysis.

### 🧩 Our Approach: Weighted Average Heuristic for Estimating Vulnerability Lifetimes

#### 🧠 Motivation
Finding the *exact* commit that introduced a vulnerability (VCC) is difficult and often unreliable for large datasets.  
Instead of identifying one exact commit, the authors aimed to **approximate the introduction time** of a vulnerability — good enough to estimate its lifetime accurately.

---

#### ⚙️ Method Overview
They **repurposed and modified** the `VCCFinder` heuristic to estimate *how long* code was vulnerable rather than pinpointing an exact VCC.

**Main Steps:**
1. **Use `git blame`** on the fixing commit to identify potential VCCs (commits responsible for vulnerable lines).  
2. **Filter out irrelevant data:**
   - Ignore **test files**, **non–C/C++ files**, **comments**, and **empty lines**.
3. **Blame logic for identifying likely VCCs:**
   - **Removed lines** → always blamed.
   - **Added code blocks (≥2 lines)** → blame lines *before and after* the addition (unless it’s a function definition).
   - **Single-line additions** → blame if they contain control-flow keywords  
     (`if`, `else`, `goto`, `return`, `sizeof`, `break`, `NULL`) or **function calls**.
4. **Calculate estimated introduction date (`dₕ`):**
   - Take the **weighted average** of all blamed commit dates.  
   - Weight = number of times each commit was blamed.  
   - Convert all dates into “days since Jan 1, 1900” for consistent calculation.

---

#### 📊 Statistical Improvement over Older Methods
Compared to Li & Paxson’s lower-bound approach:
- **Mean error reduced by 66%.**
- **Average lifetime overestimated by only ~117 days** (vs. underestimation of ~347 days previously).
- **Error distribution is symmetric**, meaning over- and underestimates cancel out.
- When using **20+ CVEs**, the average result is statistically reliable.

**Error margins (95% confidence):**
| Sample Size | Margin of Error (± days) |
|--------------|--------------------------|
| 20 samples   | ±395 days                |
| 50 samples   | ±250 days                |
| 100 samples  | ±176 days                |

---

#### 🧮 Why It’s Statistically More Reliable
- Follows the **Central Limit Theorem** — averaging multiple samples reduces noise.  
- Treats individual errors as **independent random variables**, so they cancel out on average.  
- Requires a **minimum of 20 samples** to ensure statistical validity, though most analyses used 60+.  
- Performs consistently across **different projects** (Linux Kernel, Chromium, Apache).

---

#### 🚧 Concerns Addressed
- **Overfitting / Project Dependency:**  
  Older “constant offset” corrections failed when applied to different projects; the new method works **without tuning**.
- **Small-sample unreliability:**  
  Large standard deviation (>900 days) in individual samples led them to use mean values across multiple CVEs.
- **Cross-project generalization:**  
  The method works consistently across projects, unlike additive-correction approaches that required recalibration.
- **Trend and Distribution Analysis:**  
  The heuristic preserved the **shape of lifetime distributions** and accurately reflected **trends over time**, confirmed via:
  - Similar **linear trends** between estimated and ground truth lifetimes.
  - Comparable **histogram and exponential fits**.

---

#### ✅ **Conclusion**
The **weighted average heuristic** provides:
- A **scalable**, **tuning-free**, and **statistically robust** way to estimate vulnerability lifetimes.
- Significant improvement over older `git blame`-based approaches.
- Reliable performance across projects, allowing for accurate **trend** and **distribution** analysis at scale.